# Proyecto 1: Clock App

Los temas más importantes que trataremos en este proyecto son:
* La sintáxis básica del lenguaje Kivy.
* Llamar y ubicar componentes de Kivy.
* Hacer que una aplicación esté pendiente de los eventos.
* Cargar diferentes fuentes y dar formato al texto.

Ahora que sabemos como crear la estructura base de una app crearemos nuestra primera aplicación seria. Se trata de un reloj con cronómetro. Para comenzar crearemos la estructura básica como ya sabemos hacerlo. El programa principal en Python (llamado `main.py`) y el archivo auxiliar en lenguaje KV llamado clock.kv, ya que nuestra aplicación se llamará ClockApp. Recordemos que el fichero kv debe llevar de nombre el mismo que la app pero quitándole la App final y comenzando con minúscula.

El archivo principal será:

In [ ]:
# File: main.py
from kivy.app import App

class ClockApp(App):
    pass

if __name__ == '__main__':
    ClockApp().run()

Y el archivo `clock.kv`:

In [ ]:
# File: clock.kv
BoxLayout:
    orientation: 'vertical'
        
    Label:
        text: '00:00:00'

Hasta ahora la aplicación se comporta como nuestra app anterior "Hello World!". El contenedor `BoxLayout` permite que dos o más widgets que tenga en su interior puedan coexistir uno al lado del otro. Es como un recipiente y los widgets que tienga en su interior diremos que son hijos suyos. Si solo contiene un único widget, como en nuestro caso, `BoxLayout` llenará todo el espacio disponible en la pantalla con él. Como `Label` es el widget raíz ha llenado toda la ventana de la aplicación con la etiqueta.

## Haciendo que el reloj funcione

Kivy es un lenguaje basado en eventos, es decir deberá poder ser capaz de estar atento a los posibles eventos producidos por el usuario. Por este motivo Kivy tendrá un bucle principal y después de realizar alguna tarea deberá volver a él para seguir "escuchando" posibles eventos. Por ese motivo no debemos hacer nunca que una terea específica de la aplicación entre en un bucle infinito porque eso hará que nuestra aplicación no responda a los eventos del usuario y se "cuelgue". Por eso necesitamos que una arquitectura basada en eventos esté siempre lista para responder a eventos de distintos tipos, como entradas del usuario, eventos de red o finalizaciones.

Uno de los eventos más comunes a los que los programas suelen escuchar es `App.on_start`. Un método con este nombre se define en la clase de la aplicación y será llamado tan pronto como la aplicación sea llamada. Otro evento que usaremos comunmente será `on_press`, que se dispara cuando el usuario presiona sobre un botón.

Para acceder al widget `Label` que escribe el valor del tiempo le daremos un identificador único (`id`). Posteriormente nos será más fácil acceder a los widgets a través de estos identificadores.

Modifiquemos nuestro archivo `clock.kv` agregando:

In [ ]:
Label:
    id: time

Ahora podremos accder a `Label` desde nuestro código directamente usando la notación `root.ids.time` (`root` en nuestro caso es `BoxLayout`).

Para actualizar la hora agregaremos a nuestra clase `ClockApp` el método `update` que se verá así:

In [ ]:
def update(self,intervalo):
    self.root.ids.time.text = strftime('%H:%M:%S')

La función `strftime` viene en el módulo `time` de la biblioteca estándar, por lo tanto deberemos importarla agregando a las primeras líneas de nuestro código:

In [ ]:
from time import strftime

Haremos que la función `update` se ejecute cada segundo para actualizar el reloj, como el reloj marca hasta el segundo no será necesario actualizarlo con más frecuencia. Para actualizar el reloj contamos con un método de la clase Clock que viene con Kivy, se trata del método `schedule_time` que tiene como argumentos la función que se quiere lanzar y el período o intervalo de tiempo para hacerlo (en este caso 1 corresponde a 1 segundo). Deberemos llamar a este método cuando la aplicación comience a través de la función `on_start`:

In [ ]:
def on_start(self):
    Clock.schedule_interval(self.update, 1)

Para poder usar este método deberemos importar la clase `Clock` de Kivy poniendo entre las líneas de importación:

In [ ]:
from kivy.clock import Clock

Si ejecutamos nuestra aplicación ahora veremos como... ¡el reloj funciona!

## Disposición de los objetos: Layout

Para disponer los widgets en la pantalla, Kivy provee una serie de clases `Layout`. `Layout` es una subclase de `Widget` y sirve como un contenedor para otros widgets.Para nuestra aplicación queremos una disposición sencilla, como la de la figura de abajo.

<img src="layout.png">

Para nuestra aplicación ya hemos utilizado el `BoxLayout`, que es en esencia una grilla unidimensional. Como por ahora solo hemos puesto dentro un único elemento éste no se ve afectado por otra cosa.

Los layouts de Kivy intentan llenar toda la pantalla, por lo que nuestra aplicación se adaptará al cambio de medida y orientación de la pantalla de manera automática.

Si añadimos otro BoxLayout a nuestra pantalla, tomará la mitad del tamaño de la misma, dependiendo de la orientación: un boxlayout vertical crecerá de abajo hacia arriba, y una horizontal de izquierda a derecha.

Mirando la dispocición que queremos adoptar seguramente habrás adivinado que si queremos alinear los botones horizontalmente deberemos incluir dentro del primer boxlayout otro horizontal. Los layouts y los widgets pueden anidarse unos dentro de otros de manera reativa para lograr interfaces complejas.

## Finalizando el Layout

Ahora agregaremos los widgets a nuestro reloj. Ya teníamos un `BoxLayout` vertical con una etiqueta (`Label`) dentro. Ahora agregaremos un `Boxlayout` horizontal que contendrá nuestros dos botones en su interior y una segunda etiqueta que mostrará nuestro cronómetro. El código que aparece abajo es autoexplicativo.

In [ ]:
BoxLayout:
    orientation: 'vertical'
    
    Label:
        id: time
        text: '00:00:00'
        
    BoxLayout:
        orientation: 'horizontal'
        
        Button:
            text: 'Start'
            
        Button:
            text: 'Reset'
            
    Label:
        id: crono
        text: '00:00.00'

Si probamos este código veremos que funciona, aunque no se ve muy bonito, luego veremos como mejorar su aspecto.

Por ahora nuestro reloj solo contiene un reloj normal que mide el tiempo y el archivo principal completo hasta ahora se vería así:

In [ ]:
# File: main.py
from kivy.app import App
from kivy.clock import Clock

from time import strftime

class ClockApp(App):

    def update(self, intervalo):
        self.root.ids.time.text = strftime('%H:%M:%S')
    
    def on_start(self):
        Clock.schedule_interval(self.update, 1)

    
if __name__ == '__main__':
    ClockApp().run()

## Agregando el cronómetro

Ahora queremos agregar un cronómetro a nuestra aplicación. El cronómetro ya no es tan simple como el reloj ya que tenemos que poder pararlo, volverlo a cero (reset) y volver a arrancarlo. Para hacer nuestro propio contador de tiempo usaremos el método `Clock.schedule_interval`, que permite producir eventos separados por un intervalo de tiempo que se le indica como un argumento, como ya hemos visto en el caso del reloj común, que se actualiza cada segundo. En este caso queremos que el reloj se actualice cada centésima de segundo, ya que el cronómetro contará hasta la centésima de segundo. Para hacerlo haremos que el intervalo de la función `schedule_time` sea de 0.01 en lugar de 1. También crearemos la variable `cron_time` con un valor inicial de 0 a la que le iremos añadiendo el tiempo que pase a partir del momento en que el cronómetro empiece a funcionar. Con estos añadidos nuestro código quedará:

In [ ]:
# File: main.py
from kivy.app import App
from kivy.clock import Clock

from time import strftime

class ClockApp(App):
    cron_time = 0   # <--NUEVO: define e inicializa la variable que medirá el tiempo 

    def update(self, intervalo):
        self.root.ids.time.text = strftime('%H:%M:%S')
        self.cron_tiempo += intervalo  # <--NUEVO: suma intervalo a la variable cron_tiempo
    
    def on_start(self):
        Clock.schedule_interval(self.update, 0.01) # <--NUEVO: Cambiamos el tiempo de refresco a 1/100 seg

    
if __name__ == '__main__':
    ClockApp().run()

Ahora ya tenemos guardado el tiempo que va pasando desde que comienza a funcionar la aplicación (esto lo cambiaremos más adelante para que solo se incremente el tiempo cuando el cronómero esté en funcionamiento). Ahora veremos como hacer para que la etiqueta que corresponde al cronómetro vaya cambiando en la pantalla a medida que pasa el tiempo. 

La variable `cron_time` es una variable que cuenta el tiempo en segundos y tiene decimales, así que deberemos extraer de este número los minutos, segundos y centésimas de segundo que contiene. Para hacerlo podríamos aplicar:

In [ ]:
minutos = self.cron_time / 60
segundos = self.cron_time % 60

Esto mismo se puede hacer de manera mucho más rápida y eficiante en Python usando la función `divmod` de la siguiente manera:

In [ ]:
minutos, segundos = divmod(self.cron_time, 60)

Los minutos y segundos obtenidos de esta manera son del tipo `float` así que deberemos convertirlos en enteros haciendo `int(minutos)` y `int(segundos)`. Las centésimas de segundo las podemos obtener haciendo:

In [ ]:
centesimas = int(segundos * 100 % 100)

La función `update` tendrá ahora en cuenta las variaciones en el cronómetro.

In [1]:
def update(self, intervalo):
    self.cron_time += intervalo
    
    self.root.ids.time.text = strftime('%H:%M:%S')
    
    minutes, seconds = divmod(self.cron_time, 60)
    
    self.root.ids.crono.text = (
        '%02d:%02d.%02d' %
        (int(minutes), int(seconds),
        int(seconds * 100 % 100)))

Ahora, si ejecutamos la aplicación, veremos que el cronómetro funciona y que se actualiza correctamente cambiando la pantalla a la centésima de segundo. El problema que tenemos ahora es que el conómetro arranca a funcionar cuando arranca la aplicación y nosostros no queremos eso. Queremos que el cronómetro arranque cuando nosotros apretemos el botón "Start", para ello haremos que los botones sean operativos.

## Controlando el cronómetro

Controlar una aplicación mediante los eventos de presionar botones es muy sencillo. Todo lo que necesitaremos es crear la varaiable `cron_started` e inicializarla a `False` en el momento de definir la clase `ClockApp`. Posteriormente definiremos dos métodos de la clase que nos permitirán controlar los botones: `start_stop` y `reset`, como se ve a continuación:

In [ ]:
def start_stop(self):
    self.cron_started = not self.cron_started
    self.root.ids.start_stop.text = ('Stop' if self.cron_started == False else 'Start')
    

def reset(self):
    if self.cron_started:
        self.root.ids.start_stop.text = 'Start'
        self.cron_started = False
    self.cron_time = 0

Podría decirse que el código de las funciones son casi autoexplicativos. El método `start_stop` cambia el estado de la variable `cron_started`, lo que quiere decir que si estaba en el estado `True` la cambia a `False` y viceversa. Esto quiere decir que si el cronómetro estaba funcionando lo detiene, y si estaba parado lo arranca. Esto en la teoría, pero nosotros sabemos que la función que se encarga de actualizar el cronómetro es el método `update`, por lo que si queremos que efectivamente la aplicación arranque y se detenga cuando llamamos a la función `start_stop` debemos decirle a la función `update` que actualice el cronómetro solamente cuando la variable `cron_started` está en estado `True`. Para hacerlo debemos poner un condicional en la función `update` como se ve acontinuación:

In [ ]:
if self.cron_started == True:
    self.cron_time = self.cron_time + intervalo

El método `reset` por otra parte lo que hace es cambiar la etiqueta del botón de arrancada y parada a 'Start', poner la variable `cron_started` en `False` y el cronómetro a cero haciendo que `cron_time` tome el valor 0.

Ahora las funciones están operativas, solo nos falta que se activen cuando tocamos los botones. Para hacerlo tenemos que ir al archivo `clock.kv`, donde tenemos todas las configuraciones de gadgets. Lo primero que haremos es poner el `id: start_stop` en el primer botón y hacer que cuando se presione llame a la función `app.start_stop()`. Para el segundo botón hacemos algo similar. El código completo del archivo `clock.kv` nos quedará ahora:

In [ ]:
# File: clock.kv

BoxLayout:
    orientation: 'vertical'
    
    Label:
        id: time
        text: '00:00:00'
        
    BoxLayout:
        orientation: 'horizontal'
        
        Button:
            id: start_stop
            text: 'Start'
            on_press: app.start_stop()
            
        Button:
            id: reset
            text: 'Reset'
            on_press: app.reset()
            
    Label:
        id: crono
        text: '00:00.00'

Nuestro programa `main.py` completo hasta ahora será:

In [ ]:
# File: main.py
from kivy.app import App
from kivy.clock import Clock

from time import strftime

class ClockApp(App):
    cron_time = 0
    cron_started = False
    
    def update(self, intervalo):
        if self.cron_started == True:
            self.cron_time = self.cron_time + intervalo
    
        self.root.ids.time.text = strftime('%H:%M:%S')
    
        minutes, seconds = divmod(self.cron_time, 60)
    
        self.root.ids.crono.text = (
            '%02d:%02d.%02d' %
            (int(minutes), int(seconds),
            int(seconds * 100 % 100)))

    
    def on_start(self):
        Clock.schedule_interval(self.update, 0.01)

    def start_stop(self):
        self.root.ids.start_stop.text = ('Start' if self.cron_started else 'Stop')
        self.cron_started = not self.cron_started

    def reset(self):
        if self.cron_started:
            self.root.ids.start_stop.text = 'Start'
            self.cron_started = False
        self.cron_time = 0
    
if __name__ == '__main__':
    ClockApp().run()

## Haciendo que nuestra aplicación se vea más bonita

Nuestra aplicación ahora es completamente operativa, ahora solo nos queda ponerla más bonita cambando un poco el diseño.

### Cambiando el tipo de fuente

Para mejorar el aspecto podemos cambiar el tipo de fuente utilizada. Utilizaremos la fuente `Roboto` que es la que utiliza Android en su versión Jelly Beans. Puedes descargar los archivos de la fuente en tu ordenador desde la carpeta de archivos auxiliares del Aula Virtual.

Las modificaciones de formato las hacemos en el archivo `clock.kv`, que debería quedar así:

In [ ]:
# File: clock.kv

BoxLayout:
    orientation: 'vertical'
    
    Label:
        id: time
        text: '00:00:00'
        font_name: 'Roboto-Thin'
        font_size: 60
        markup: True
        
    BoxLayout:
        height: 90
        orientation: 'horizontal'
        padding: 20
        spacing: 20
        size_hint: (1, None)
        
        Button:
            id: start_stop
            text: 'Start'
            font_name: 'Roboto-Medium'
            font_size: 25
            bold: True
            on_press: app.start_stop()
            
        Button:
            id: reset
            text: 'Reset'
            font_name: 'Roboto-Thin'
            font_size: 25
            bold: True
            on_press: app.reset()
            
    Label:
        id: crono
        text: '00:00.[size=40]00[/size]'
        font_name: 'Roboto-Thin'
        font_size: 60
        markup: True

El problema con esta manera de trabajar es que el código resulta muy repetitivo, para cada componente debemos definir todas sus características. Seria muy bueno poder definir las características una vez y luego que los componentes hereden esas características. Eso ya existe y lo conocemos, lo llamamos "clases". Nos bastaría crear una clase para Label con todas las características. En el lenguaje kivy podemos crear una clase llamada "Etiqueta" que pertezca al gadget Label y definir todas las características una sola vez. eso lo haríamos así:

In [ ]:
<Etiqueta@Label>:
    font_name: 'Roboto-Thin'
    font_size: 60
    markup: True

Lo que pongamos entre corchetes es el nombre de la clase (en este caso Etiqueta) y lo que aparece después de @ es el gadget al que afecta (en este caso Label), ahora todas las Label tendran las características definidas por la clase y no tendremos que volver a definirlo para cada Label. ¿qué pasa si queremos que alguna label tenga características diferentes? sencillamente las definimos para esa Label, esas características sobreescribiran las de la clase solamente para esa Label.

Lo mismo podemos hacer con los botones, podemos crear la clase `<RobotButton@Button>` que defina todas las características comunes de los botones, de la siguiente manera:

In [ ]:
<RobotoButton@Button>:
    font_name: 'Roboto-Medium'
    font_size: 25
    bold: True

Ahora nuestro archivo de configuración `clock.kv` se verá libre de repeticiones y mucho más simplificado.

In [ ]:
# File: clock.kv

<Etiqueta@Label>:
    font_name: 'Roboto-Thin'
    font_size: 60
    markup: True

<RobotoButton@Button>:
    font_name: 'Roboto-Medium'
    font_size: 25
    bold: True
    background_normal: 'button_normal.png'
    background_down: 'button_down.png'
    border: (2, 2, 2, 2)


BoxLayout:
    orientation: 'vertical'
    
    Etiqueta:
        id: time
        text: '[b]00[/b]:00:00'
        
    BoxLayout:
        height: 90
        orientation: 'horizontal'
        padding: 20
        spacing: 20
        size_hint: (1, None)
        
        RobotoButton:
            id: start_stop
            text: 'Start'
            on_press: app.start_stop()
            
        RobotoButton:
            id: reset
            text: 'Reset'
            on_press: app.reset()
            
    Etiqueta:
        id: crono
        text: '00:00.[size=40]00[/size]'